In [1]:
from midiutil import MIDIFile
import rebound
import numpy as np
import matplotlib.pyplot as plt
from subprocess import call
from itertools import repeat
import PIL # reminder that this is a requirement
from scipy.misc import imread

In [2]:
class PlanetBeat():
    def __init__(self, filename, bpm, time_per_beat=None, dt=None, outer_midi_note=48, fps=30):
        try:
            call("rm -f ./tmp/*", shell=True)
        except:
            pass
        self.midi = MIDIFile(adjust_origin=True) # One track, defaults to format 1 (tempo track automatically created)
        self.filename = filename
        self.sim = rebound.Simulation.from_file(filename)
        self.sim.t = 0
        if time_per_beat:
            self.time_per_beat = time_per_beat
        else:
            self.time_per_beat = self.sim.particles[-1].P
        if not dt:
            self.sim.dt = self.sim.particles[1].P/100. # use small timestep to get transits right. This is not the bottleneck
        else:
            self.sim.dt = dt
        
        self.bpm = bpm
        self.fps = fps
        self.fig_ctr = 0
        self.time_elapsed = 0
        self.time_per_fig = 1./self.fps
        
        self.notes = self.calc_midi_notes(outer_midi_note)
        self.velocities = [100 for i in range(self.sim.N)]
        self.conjunction_notes = [12 for i in range(self.sim.N)] # C1
        self.conjunction_velocities = [100 for i in range(self.sim.N)]
        
        set_time_per_beat(self.sim, self.time_per_beat)
        self.change_tempo(bpm)
        self.fig_params = []
        self.conjunctions = []
    def calc_midi_notes(self, outer_midi_note):
        # 12 notes between octaves, freq = f0*2**(n/12). 
        # n = 12 log_2(freq/f0)
        # star, then planets from inside out
        ps = self.sim.particles
        midinotes = [0] # placeholder for star
        for p in ps[1:]:
            midinote = outer_midi_note+12*np.log(ps[-1].P/p.P)/np.log(2)
            midinotes.append(int(np.round(midinote)))
        return midinotes  
    def make_tuple(self, arg):
        N = self.sim.N
        if arg == True:
            return tuple(range(1,N))
        elif arg == False:
            return ()
        else:
            return tuple(arg)
    def integrate(self, tmax, color=True, duration=1, track=0, playtransits=True, playconjunctions=False, showplanets=True, showtransits=True, showconjunctions=True):
        playtransits = self.make_tuple(playtransits)
        playconjunctions = self.make_tuple(playconjunctions)
        showplanets = self.make_tuple(showplanets)
        showtransits = self.make_tuple(showtransits)
        showconjunctions = self.make_tuple(showconjunctions)
        
        N=self.sim.N
        ps = self.sim.particles
        yprev = np.zeros(N)
        sinthetaprev = np.zeros(N)
        while self.sim.t < tmax:
            self.sim.step()
            self.time_elapsed += self.sim.dt/self.bpm*60.
            for j in playtransits:
                if yprev[j] < 0 and ps[j].y > 0:
                    #print(self.sim.t, j)
                    #print(ps[j].index)
                    self.midi.addNote(track, ps[j].index, self.notes[j], self.sim.t, duration, self.velocities[j])
                yprev[j] = ps[j].y
            for j in playconjunctions:
                if j+1 in planets:
                    sintheta = np.sin(ps[j+1].theta-ps[j].theta)
                    if sinthetaprev[j] > 0 and sintheta < 0:
                        #print('conjunction', self.sim.t, j)
                        self.conjunctions.append((self.sim.t, j))
                        self.midi.addNote(track, N, self.conjunction_notes[j], self.sim.t, duration, self.conjunction_velocities[j]) # add to track above all planets
                    sinthetaprev[j] = sintheta
            if self.time_elapsed/self.time_per_fig > self.fig_ctr + 1:
                #print(self.time_elapsed*self.fps)
                self.fig_params.append([self.fig_ctr, self.sim.t, filename, self.time_per_beat, color, tuple(showplanets), tuple(showtransits), tuple(showconjunctions)])
                self.fig_ctr += 1
    def change_tempo(self, bpm):
        self.bpm = bpm
        self.midi.addTempo(0, self.sim.t, self.bpm) 
    def write_midi(self, filename):
        with open(filename, "wb") as f:
            self.midi.writeFile(f)

def set_time_per_beat(sim, time_per_beat): # makes sim.t run in units of the outer planet orbit = one beat
        ps = sim.particles
        sim.G = time_per_beat**2
        sim.dt /= time_per_beat
        for p in ps:
            p.vx *= time_per_beat
            p.vy *= time_per_beat
            p.vz *= time_per_beat
    
def write_png(params):
    fig_ctr, time, filename, time_per_beat, color, showplanets, showtransits, showconjunctions = params
    sim = rebound.Simulation.from_file(filename)
    sim.t=0
    set_time_per_beat(sim, time_per_beat)
    sim.integrate(time)
    ps = sim.particles
    
    lw=3
    scale=ps[1].a/5 # length scale for making dots bigger
    refsize=25*lw # this is what REBOUND uses for size of circles in call to plt.scatter

    fig = rebound.OrbitPlot(sim, figsize=(8,8), color=color, lw=lw, plotparticles=showplanets)
    ax = fig.axes[0]
    ax.axis('off')
    for i in showtransits:
        p = ps[i]
        if p.x > 0:
            ax.scatter(p.x, p.y, s=refsize*(1+5*np.exp(-np.abs(p.y)/scale)),color='k', marker='o',zorder=1)
    #if doconjunctions:
    #    nearby_conjunctions = [conjunction for conjunction in conjunctions if np.abs(conjunction[0]-time) < sim.particles[-1].P/10.]
    #    for conjunction in nearby_conjunctions:
    #        p = ps[conjunction[1]]
    #        ax.plot([0, p.x], [0,p.y], lw=5)
    
    bkg = imread('images/US_background_image.png')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    ax.imshow(bkg, zorder=0, extent=xlim+ylim)
    #fig.savefig('test.png')
    fig.savefig('tmp/pngs/{0:0=5d}.png'.format(fig_ctr))
    plt.close(fig)  

# Pick colors from

http://stackoverflow.com/questions/22408237/named-colors-in-matplotlib

In [3]:
filename = "binaries/trappist.bin"
colors = ['fuchsia', 'chartreuse', 'yellow', 'red', 'cyan', 'darkviolet', 'whitesmoke']
pb = PlanetBeat(filename, bpm=30, outer_midi_note=48) # notes: http://subsynth.sourceforge.net/midinote2freq.html
planets = [1,2,3,4,5,6,7]

pb.integrate(1, duration=1, color=colors[::-1])
    
midiname = "midi"
pb.write_midi("./tmp/"+midiname+".mid")

/Users/dtamayo/Documents/workspace/rebound/rebound/simulation.py:374: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/Users/dtamayo/Documents/workspace/rebound/rebound/simulation.py:374: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)


In [4]:
%%time
import warnings
warnings.filterwarnings("ignore")

call("rm -f tmp/pngs/*", shell=True)
pool = rebound.InterruptiblePool()
res = pool.map(write_png, pb.fig_params)

CPU times: user 21 ms, sys: 29.6 ms, total: 50.5 ms
Wall time: 6.51 s


In [5]:
call("timidity -Ow ./tmp/{0}.mid -o ./tmp/{0}.wav --preserve-silence".format(midiname), shell=True)
call("ffmpeg -t {0} -i ./tmp/{1}.wav ./tmp/{1}cut.wav".format(pb.time_elapsed, midiname), shell=True)
moviename = "colors.mp4"
fps = 30
try:
    call("rm -f {0}".format(moviename), shell=True)
except:
    pass
call("ffmpeg -r {0} -i tmp/pngs/%05d.png -i tmp/{1}cut.wav -c:v libx264 -pix_fmt yuv420p -c:a libvo_aacenc -b:a 192k -shortest {2}".format(fps, midiname, moviename), shell=True)

0

In [6]:
call("open "+moviename, shell=True)

0

In [8]:
import matplotlib.colors as mplcolors

def trycolor(color):
    try:
        hexcolor = mplcolors.cnames[color]
    except KeyError:
        raise AttributeError("Color passed to OrbitPlot not recognized in matplotlib.")


In [22]:
trycolor('slategrey')